### Python Debugging with __`pdb`__
* __`pdb`__ can be run
  * from the commandline
  * from within the Python interpreter
  * from within your Python program
  * from within Jupyter!
* important to distinguish between using __`pdb`__ to run or step through a program, vs. using __`pdb`__ in post-mortem mode, to determine why a program crashed
  * we will demonstrate both...

### Let's first run from within Jupyter...
* In order to do that, it's helpful to be familiar with a couple of Jupyter's "magic" commands
   * __`%debug`__ = start __`pdb`__ after a crash
   * __`%pdb`__ = toggle automatic calling of __`pdb`__ after a crash
   * __`%run`__ = run a named file inside of Jupyter
     * -d = run your program under the control of __`pdb`__
     * -t = output approximate timing information
     * -N__n__ (used with -t) = run the program __n__ times

### First we'll create a crash and do a post-mortem debug...
* Here's some code which computes prime numbers using the Sieve of Eratosthenes
* Using random(), I've hard-coded a bug which occurs 0.5% of the time, causing a crash


In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [3]:
from random import random
limit = 100
numbers = list(range(2, limit + 1))
primes = []

while numbers:
    candidate = numbers[0]
    primes.append(candidate)
    for num in range(candidate, limit + 1, candidate):
        if num in numbers:
            rval = random()
            if rval > 0.99:
                raise ValueError
            else:
                numbers.remove(num)

print(primes)

ValueError: 

> <ipython-input-3-f7f74f07a6e0>(13)<module>()
     11             rval = random()
     12             if rval > 0.99:
---> 13                 raise ValueError
     14             else:
     15                 numbers.remove(num)

ipdb> print(rval)
0.9906660991138615
ipdb> p num
67
ipdb> bt
> <ipython-input-3-f7f74f07a6e0>(13)<module>()
     11             rval = random()
     12             if rval > 0.99:
---> 13                 raise ValueError
     14             else:
     15                 numbers.remove(num)

ipdb> n


### Basic __`pdb`__ commands
 * __c(ontinue)__ = continue running until the next breakpoint
 * __s(tep)__ = step, i.e., move one line ahead–stops inside a called function
 * __n(ext)__ = next, i.e., move one line ahead–executes called functions at (nearly) full speed, only stopping at the next line in the current function
 * __l(ist)__ = list code
   * current line in the current frame is indicated by `->`
   * if an exception is being debugged, the line where the exception was originally raised or propagated is indicated by `>>`, if it differs from the current line
 * __b(reak) `func`__ = breakpoint inside function `func`
 * __b(reak) `num`__ = breakpoint at line number `num`
 * __t(break)__ = same as __b__, but breakpoint is removed after first hit
 * __cl(ear)__ = clear breakpoints
 * __unt(il) `num`__ = continue execution until a line number >= `num` is reached
 * __r(eturn)__ = continue execution until current function returns
 * __j(ump) `num`__ = jump back to line `n` (can't jump into a loop)
 * __p `expression`__ = print `expression`
 * __pp `expression`__ = pretty print `expression`
 * __q(uit)__ = quit

### Next, let's run an "outside" script directly from within Jupyter
* and then let's try running it with debugging turned on

In [1]:
%%file data/primes.py
from random import random

limit = 100
numbers = list(range(2, limit + 1))
primes = []

while numbers:
    candidate = numbers[0]
    primes.append(candidate)
    for num in range(candidate, limit + 1, candidate):
        if num in numbers:
            numbers.remove(num)
print(primes)



Writing data/primes.py


In [3]:
%run data/primes.py

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [4]:
# might have to restrt the kernel before doing this
%run -d data/primes.py

Breakpoint 1 at /Users/rick446/src/arborian-classes/src/data/primes.py:1
NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> /Users/rick446/src/arborian-classes/src/data/primes.py(1)<module>()
1---> 1 from random import random
      2 
      3 limit = 100
      4 numbers = list(range(2, limit + 1))
      5 primes = []

ipdb> q


### To run from the command line, we need to jump out of the notebook for a moment...
* The command we'll use is

        python3 -m pdb script.py

### Now that we're back, we can run from the interpreter...
* This will allow us to test a module without having to do the save/run/import cycle

In [7]:
class PdbTester():
    def __init__(self, count):
        self.count = count

    def doit(self):
        for i in range(self.count):
            print(i)

In [8]:
from mypdb import PdbTester
import pdb
pdb.run('PdbTester(3).doit()')

> <string>(1)<module>()
(Pdb) list
[EOF]
(Pdb) s
--Call--
> /Users/rick446/src/Classes/IntermediatePython/mypdb.py(2)__init__()
-> def __init__(self, count):
(Pdb) list
  1  	class PdbTester():
  2  ->	    def __init__(self, count):
  3  	        self.count = count
  4  	
  5  	    def doit(self):
  6  	        for i in range(self.count):
  7  	            print(i)
[EOF]
(Pdb) r
--Return--
> /Users/rick446/src/Classes/IntermediatePython/mypdb.py(3)__init__()->None
-> self.count = count
(Pdb) s
--Call--
> /Users/rick446/src/Classes/IntermediatePython/mypdb.py(5)doit()
-> def doit(self):
(Pdb) list
  1  	class PdbTester():
  2  	    def __init__(self, count):
  3  	        self.count = count
  4  	
  5  ->	    def doit(self):
  6  	        for i in range(self.count):
  7  	            print(i)
[EOF]
(Pdb) bt
  /usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/bdb.py(431)run()
-> exec(cmd, globals, locals)
  <string>(1)<module>()
> /Users/rick446/src/C

### What if we have a program where the bug doesn't occur until the program has been running for a while?
* In a case such as this, we're going to want to invoke __`pdb`__ from *within* our program...
* ...but not from within the notebook, so we'll once again drop out of the notebook to examine the following code:

In [9]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    import pdb
    pdb.set_trace() # causes the program to stop here, in the debugger
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

do a lot more stuff
and now we get to the bug...
> <ipython-input-9-bff64036bdf8>(10)<module>()
-> print('buggy line')
(Pdb) list
  5  	x = 1
  6  	print('and now we get to the bug...')
  7  	if x == 1: # this is the buggy case
  8  	    import pdb
  9  	    pdb.set_trace() # causes the program to stop here, in the debugger
 10  ->	    print('buggy line')
 11  	    y = 1
 12  	else:
 13  	    y = x ** 2 # no debugging in this case
 14  	print(x)
[EOF]
(Pdb) p x
1
(Pdb) c
buggy line
1


In [10]:
for num in range(1, 1000): # do something
    pass

print('do a lot more stuff')
x = 1
print('and now we get to the bug...')
if x == 1: # this is the buggy case
    import ipdb
    ipdb.set_trace() # causes the program to stop here, in the debugger
    print('buggy line')
    y = 1
else:
    y = x ** 2 # no debugging in this case
print(x)

do a lot more stuff
and now we get to the bug...
> <ipython-input-10-b0947057f198>(10)<module>()
      9     ipdb.set_trace() # causes the program to stop here, in the debugger
---> 10     print('buggy line')
     11     y = 1

ipdb> q


BdbQuit: 

> /usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/bdb.py(67)dispatch_line()
     65         if self.stop_here(frame) or self.break_here(frame):
     66             self.user_line(frame)
---> 67             if self.quitting: raise BdbQuit
     68         return self.trace_dispatch
     69 

ipdb> q


In [11]:
import math
class Calculator(object):
    
    def __init__(self):
        self._acc = 0
        self._calculations = []
        
    def add(self, a, b=None):
        if b is None:
            self._acc = self._acc + a
            calc = f'+ {a}'
        else:
            self._acc = a + b
            calc = f'{a} + {b}'
        self._calculations.append(calc)
        return self._acc

    def sub(self, a, b=None):
        if b is None:
            self._acc = self._acc - a
            calc = f'- {a}'
        else:
            self._acc = a - b
            calc = f'{a} - {b}'
        self._calculations.append(calc)
        return self._acc

    def mult(self, a, b=None):
        if b is None:
            self._acc = self._acc * a
            calc = f'* {a}'
        else:
            self._acc = a * b
            calc = f'{a} * {b}'
        self._calculations.append(calc)
        return self._acc

    def div(self, a, b=None):
        if b is None:
            self._acc = self._acc / a
            calc = f'/ {a}'
        else:
            self._acc = a / b
            calc = f'{a} / {b}'
        self._calculations.append(calc)
        return self._acc

    def pow(self, a, b=None):
        if b is None:
            self._acc = self._acc ** a
            calc = f'** {a}'
        else:
            self._acc = a ** b
            calc = f'{a} ** {b}'
        self._calculations.append(calc)
        return self._acc

    def log(self, a=None):
        if a is None:
            self._acc = math.log(self._acc)
            calc = f'log'
        else:
            self._acc = math.log(a)
            calc = f'log({a})'
        self._calculations.append(calc)
        return self._acc
    
    def ac(self):
        self._acc = 0
        self._calculations = []

    def showcalc(self):
        for calc in self._calculations:
            print(calc)

In [12]:
c = Calculator()

In [13]:
c.add(1,2)

3

In [14]:
def sum_numbers(c, numbers):
    import ipdb; ipdb.set_trace()
    c.add(0,0)
    for num in numbers:
        result = c.add(num)
    return result

In [15]:
sum_numbers(Calculator(), [1,45,2,5,10])

> <ipython-input-14-ad09e68e33e9>(3)sum_numbers()
      2     import ipdb; ipdb.set_trace()
----> 3     c.add(0,0)
      4     for num in numbers:

ipdb> list
      1 def sum_numbers(c, numbers):
      2     import ipdb; ipdb.set_trace()
----> 3     c.add(0,0)
      4     for num in numbers:
      5         result = c.add(num)
      6     return result

ipdb> p numbers
[1, 45, 2, 5, 10]
ipdb> c


63

# Lab

Open [PDB Lab][pdb-lab]

[pdb-lab]: ./pdb-lab.ipynb